# Relatório da disciplina de Algebra Linear Computacional (CKP8122)
 - Aluno: Madson Luiz Dantas Dias
 - Professor: Creto Augusto Vidal

## Lista de implementações
### básico
 1. Classe Matriz
 2. Multiplicação de vetor-matriz e matriz-vetor

### métodos diretos
 3. eliminação de Gauss
 4. decomposição LU
 5. decomposição de Cholesky
  
### métodos iterativos
 6. Jacobi
 7. Gauss-Seidel
 8. successive over-relaxation
 9. gradientes conjugados

### métodos de mínimos quadrados
 10. Gram-Schmidt
 11. decomposição QR usando householder
 12. decomposição QR usando Jacobi

### autovalores e autovetores de matrizes
 13. método da potência (regular, inverso e com deslocamento)
 14. método das transformações de similaridade (Householder + QR)
 
### final
 15. singular value decomposition

In [1]:
from Matrix import *

## Básico
### Classe Matriz
### Multiplicação  de vetor-matriz e matriz-vetor

#### Definição
Dada duas matrizes $\mathbf{A}\in\mathcal{R}^{m\times n}$ e $\mathbf{B}\in\mathcal{R}^{n\times p}$, a multiplicação de $\mathbf{A}$ por $\mathbf{B}$ é representada por outra matriz $\mathbf{C} = \mathbf{A}\mathbf{B} \in \mathcal{R}^{m \times p}$, em que cada elemento é dado pela seguinte equação

\begin{equation}
    c_{ij} = \sum_{k=1}^{n} a_{ik}b_{kj}, ~~ i = 1,2,\dots,m ~~ e ~~ j=1,2,\dots,p
\end{equation}


#### Código python
```Python
def __mul__(self, value):
    result = copy.deepcopy(self)
    if type(value) in [int, float]:
        for i in range(self.shape[0]):
            for j in range(self.shape[1]):
                result[i,j] = result[i,j] * value
    elif type(value) == Matrix and self.shape[1] != value.shape[0]:
        print("Error: Matrices with Incompatible Dimensions.")
        result = None
    else:
        result = zeros((self.shape[0], value.shape[1]))
        if self.shape[0] == 1 and value.shape[1] == 1:
            result = 0
            for i in range(self.shape[1]): result = result + self[0,i] * value[i,0]
        else:
            result = zeros((self.shape[0], value.shape[1]))
            for i in range(self.shape[0]):
                for j in range(value.shape[1]):
                    result[i,j] = self[i,:] * value[:,j]
    return result
```

In [30]:
A = Matrix([[1,2,3],[4,5,6],[7,8,9]])
B = Matrix([[10,6,9],[-1,7,9],[4,6,1]])
print(A)
print('*')
print(B)
print('=')
print(A*B)


|    1.00     2.00     3.00 |
|    4.00     5.00     6.00 |
|    7.00     8.00     9.00 |

*

|   10.00     6.00     9.00 |
|   -1.00     7.00     9.00 |
|    4.00     6.00     1.00 |

=

|   20.00    38.00    30.00 |
|   59.00    95.00    87.00 |
|   98.00   152.00   144.00 |



## Métodos diretos
### Eliminação de Gauss

#### Definição
Consideremos o sistema $\mathbf{A}\boldsymbol{x}=\boldsymbol{b}$ em que $\mathbf{A}$ é uma matriz quadrada $n \times n$.

O objetivo do método de eliminação de Gauss é eliminar os elementos $a_{ij}$ de forma a obter um sistema equivalente com uma matriz triangular superior. Depois bastará usar substituições sucessivas para chegarmos à solução pretendida. Este método pode ser sumarizado nas seguintes etapas:

 1. obtenção da matriz aumentada $[\mathbf{A} | \boldsymbol{b}]$ do sistema
 2. transformação da matriz aumentada $[\mathbf{A} | \boldsymbol{b}]$ em $[\bar{\mathbf{A}} | \bar{\boldsymbol{b}}]$, onde $\bar{\mathbf{A}}$ é uma matriz triangular superior.
 3. resolver o sistema linear $[\bar{\mathbf{A}} | \bar{\boldsymbol{b}}]$ usando retro substituição
 

O processo de transformação na matriz aumentada é baseado no pivoteamento da matriz de coeficientes $\mathbf{A}$. Primeiro, defini-se um pivô $a_{ii}$ e aplica-se as seguintes operações:
\begin{equation}
    \begin{split}
        &1.\quad L_{i} = \frac{L_{i:}}{a_{ii}},\\
        &2.\quad L_{k} = L_{k} + \alpha L_{i},\quad\forall k = i+1,\dots,n,\:\:\alpha = -\frac{a_{ki}}{a_{ii}}\\
    \end{split}
\end{equation}
em que $L_{i}$ é a $i$-ésima linha da matriz composta por $[\mathbf{A}|\boldsymbol{b}]$. Este processo é realizado para todas as linhas de $[\mathbf{A}|\boldsymbol{b}]$.

Depois de terminado o processo de redução a diante, o processo de retro-substituição é realizado da seguinte forma:
\begin{equation}
    x_i = b_i - \sum_{j=n}^{i+1} a_{ij}x_j,\quad\forall i=\dots,n.
\end{equation}

O algoritmo em python para o processo de eliminação de Gauss segue abaixo.
 
#### Código Python
##### Eliminação de Gauss
```Python
def gauss_elimination(self, b):
    Ab_ = self.concat(b)
    N, M = Ab_.shape
    # phase 1: convert A to a superior triangular matrix
    for n in range(N):
        pivot = Ab_[n,n]
        for i in range(n+1,N):
            f = Ab_[i,n] / pivot
            Ab_[i,:] = Ab_[i,:] - (Ab_[n,:] * f)
    # phase 2: solve the system using back substituition
    return Ab_.back_substituition()

```
##### Retro substituição
```Python
def back_substituition(self):
    N, M = self.shape
    # vector of soluctions
    x = zeros((1,N))

    # auxiliary matrices
    A = self[:,:-1]
    b = self[:,-1]

    if sum([A[i,i] == 0 for i in range(N)]) == 0:
        x[0,N-1] = b[N-1,0] / A[N-1,N-1]
        for j in range(N-2,-1,-1):
            x[0,j] = (b[j,0] - A[j,j+1:].dot(x[0,j+1:]).sum(axis=1).to_number() ) / A[j,j]
        return x
    else:
        print("the system not can solved by back substituition.")
        return False
```

In [9]:
A = Matrix([[ 1,-1, 2],
            [ 2, 1,-1],
            [-2,-5, 3]])
b = Matrix([[ 2],
            [ 1],
            [ 3]])

x = A.gauss_elimination(b)

print("x = ")
print(x)

x = 

|    1.00    -1.00    -0.00 |



### Decomposição LU
#### Definição
O objetivo da decomposição LU é decompor uma determinada matriz de coeficientes $\mathbf{A}\in\mathcal{R}^{n\times n}$ como o produto $\mathbf{L}\mathbf{U}$ de uma matriz $\mathbf{L}\in\mathcal{R}^{n\times n}$ triangular inferior e uma matriz $\mathbf{U}\in\mathcal{R}^{n\times n}$ triangular superior, tal que

\begin{eqnarray}
\mathbf{A}\boldsymbol{x} &=& \boldsymbol{b} \\
(\mathbf{L}\mathbf{U})\boldsymbol{x} &=& \boldsymbol{b} \\
\mathbf{L}(\mathbf{U}\boldsymbol{x}) &=&\boldsymbol{b} \\
\mathbf{L} \boldsymbol{y} = \boldsymbol{b} \quad & \text{ e }& \quad \mathbf{U}\boldsymbol{x}=\boldsymbol{y}
\end{eqnarray}

Teoricamente, a fatoração LU é equivalente ao método da eliminação de Gauss. Na prática, na fatoração LU guardamos os multiplicadores usados para transformar $\mathbf{A}$ numa matriz triangular superior $\mathbf{U}$.

#### Código Python
```Python
def lu_decomposition(self):
    N = self.shape[0]
    L = eye(N)
    U = copy.deepcopy(self)
    for n in range(N):
        L[n+1:,n] = U[n+1:,n] * (1/U[n,n])
        for l  in range(n+1,N):
            U[l,:] = U[l,:] - (U[n,:] * L[l,n])
    return L, U
```

In [8]:
L, U = A.lu_decomposition()

print("L = ")
print(L)
print("U = ")
print(U)
print("L*U = ")
print(L * U)

L = 

|    1.00     0.00     0.00 |
|    2.00     1.00     0.00 |
|   -2.00    -2.33     1.00 |

U = 

|    1.00    -1.00     2.00 |
|    0.00     3.00    -5.00 |
|    0.00    -0.00    -4.67 |

L*U = 

|    1.00    -1.00     2.00 |
|    2.00     1.00    -1.00 |
|   -2.00    -5.00     3.00 |



### Decomposição de Cholesky

#### Definição

O objetivo da decomposição de Cholesky é decompor uma determinada matriz definida positiva de coeficientes $\mathbf{A}\in\mathcal{R}^{n\times n}$ como o produto $\mathbf{L}\mathbf{L}^T$ de uma matriz $\mathbf{L}\in\mathcal{R}^{n\times n}$  triangular inferior com elementos da diagonal principal estritamente positivos, tal que

\begin{eqnarray}
l_{kk} &=& \sqrt{a_{kk} - \sum_{i=1}^{k-1} l_{ki}^2 }, \\
l_{kj} &=& \frac{a_{jk} - \sum_{i=1}^{k-1} l_{ji} l_{ki}}{l_{kk}}, ~~ j=k+1,k+2\dots, n
\end{eqnarray}

#### Código Python
```Python
def chol_decomposition(self):
    N = self.shape[0]
    L = zeros(N)
    for i in range(N):
        L[i,i] = pow(- (L[i,:] * L[i,:].transpose()) + self[i,i], 1/2)
        for j in range(i+1,N):
            L[j,i] = (- (L[i,:] * L[j,:].transpose()) + self[j,i]) / L[i,i]
    return L
```

In [11]:
A = Matrix([[ 4, 2,-4],
            [ 2,10, 4],
            [-4, 4, 9]])

L = A.chol_decomposition()

print("L = ")
print(L)
print("L*L^T = ")
print(L * L.transpose())

L = 

|    2.00     0.00     0.00 |
|    1.00     3.00     0.00 |
|   -2.00     2.00     1.00 |

L*L^T = 

|    4.00     2.00    -4.00 |
|    2.00    10.00     4.00 |
|   -4.00     4.00     9.00 |

